# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:


import cv2
import os
import numpy as np
import face_recognition
from ultralytics import YOLO
import mediapipe as mp
import threading
import speech_recognition as sr
import pywhatkit
from playsound import playsound
import warnings
import datetime
import time
import logging

warnings.filterwarnings("ignore")
logging.basicConfig(filename=r'C:\Users\sadik\PyCharmMiscProject\patient_room_monitoring.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

RECEIVER_PHONE = "+919345531046"  # Replace with actual phone number

def send_whatsapp_alert(message):
    try:
        now = datetime.datetime.now()
        timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
        alert_message = (
            f"*** Patient Room Monitoring Alert ***\n\n"
            f"Timestamp: {timestamp}\n"
            f"Alert Details: {message}\n\n"
            f"Kindly address this situation promptly to ensure patient safety and well-being.\n"
            f"Thank you."
        )
        print(f"Sending WhatsApp message:\n{alert_message}")
        logging.info(f"Sending WhatsApp message: {alert_message}")
        pywhatkit.sendwhatmsg_instantly(RECEIVER_PHONE, alert_message, wait_time=10, tab_close=True)
    except Exception as e:
        print(f"Failed to send WhatsApp message: {e}")
        logging.error(f"Failed to send WhatsApp message: {e}")

def trigger_alarm(message, sound_file=None):
    print(f"ALARM: {message}")
    logging.info(f"ALARM: {message}")
    send_whatsapp_alert(message)
    if sound_file:
        threading.Thread(target=playsound, args=(sound_file,), daemon=True).start()

def adjust_gamma(image, gamma=0.5):
    """Apply gamma correction to the image to enhance contrast in low-light conditions."""
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# --- Night Mode Functions ---
def detect_low_light(frame, threshold=60):
    """Check if the frame is in low-light condition."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    avg_brightness = np.mean(gray)
    return avg_brightness < threshold

def apply_night_mode(frame):
    """Enhance brightness and contrast for night conditions."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = np.clip(v + 40, 0, 255)  # Increase brightness
    s = np.clip(s - 30, 0, 255)  # Reduce saturation
    hsv = cv2.merge((h, s, v))
    frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    frame = adjust_gamma(frame, gamma=0.6)  # Increase contrast
    return frame

# Load YOLOv8 model
model = YOLO(r"C:\Users\sadik\PyCharmMiscProject\yolov8n.pt")

# Load known faces
known_faces_dir = r"C:\\Users\\sadik\\PyCharmMiscProject\\known_faces"
known_face_encodings = []
known_face_names = []

for filename in os.listdir(known_faces_dir):
    if filename.endswith((".jpg", ".png")):
        image_path = os.path.join(known_faces_dir, filename)
        image = face_recognition.load_image_file(image_path)
        encoding = face_recognition.face_encodings(image)
        if encoding:
            known_face_encodings.append(encoding[0])
            known_face_names.append(os.path.splitext(filename)[0])
        else:
            logging.warning(f"No face encoding found in {filename}")

# MediaPipe hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)

def count_fingers(hand_landmarks):
    tips_ids = [4, 8, 12, 16, 20]
    fingers = []
    if hand_landmarks.landmark[tips_ids[0]].x < hand_landmarks.landmark[tips_ids[0] - 1].x:
        fingers.append(1)
    else:
        fingers.append(0)
    for id in range(1, 5):
        if hand_landmarks.landmark[tips_ids[id]].y < hand_landmarks.landmark[tips_ids[id] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)
    return fingers.count(1)

# Speech recognition
recognizer = sr.Recognizer()
mic = sr.Microphone()

ALARM_SOUNDS = {
    "call_nurse": r"C:\\Users\\sadik\\PyCharmMiscProject\\alarm_sounds\\call_nurse.mp3",
    "need_water": r"C:\\Users\\sadik\\PyCharmMiscProject\\alarm_sounds\\need_water.mp3",
    "call_family": r"C:\\Users\\sadik\\PyCharmMiscProject\\alarm_sounds\\call_family.mp3",
    "cancel_request": r"C:\\Users\\sadik\\PyCharmMiscProject\\alarm_sounds\\cancel_request.mp3",
    "unknown_alert": r"C:\\Users\\sadik\\PyCharmMiscProject\\alarm_sounds\\unknown_alert.mp3",
    "crowd_alert": r"C:\\Users\\sadik\\PyCharmMiscProject\\alarm_sounds\\crowd_alert.mp3"
}

def listen_for_voice_commands():
    with mic as source:
        print("Adjusting for ambient noise... Please wait.")
        logging.info("Adjusting for ambient noise")
        recognizer.adjust_for_ambient_noise(source, duration=5)
        print("Listening for voice commands...")
        logging.info("Listening for voice commands")

        while True:
            try:
                recognizer.dynamic_energy_threshold = True
                recognizer.energy_threshold = 300
                recognizer.pause_threshold = 1.0
                recognizer.non_speaking_duration = 0.5

                audio = recognizer.listen(source, timeout=8, phrase_time_limit=10)
                try:
                    command = recognizer.recognize_google(audio, language='en-US').lower()
                    print(f"Voice Command Detected: {command}")
                    logging.info(f"Voice Command Detected: {command}")

                    if any(phrase in command for phrase in ["call nurse", "nurse", "help"]):
                        trigger_alarm("Voice Command: Call Nurse", ALARM_SOUNDS["call_nurse"])
                    elif any(phrase in command for phrase in ["need water", "water", "thirsty"]):
                        trigger_alarm("Voice Command: Need Water", ALARM_SOUNDS["need_water"])
                    elif any(phrase in command for phrase in ["call family", "family", "contact family"]):
                        trigger_alarm("Voice Command: Call Family", ALARM_SOUNDS["call_family"])
                    elif any(phrase in command for phrase in ["cancel", "stop", "abort"]):
                        trigger_alarm("Voice Command: Cancel Request", ALARM_SOUNDS["cancel_request"])
                    else:
                        print(f"Unrecognized command: {command}")
                        logging.warning(f"Unrecognized command: {command}")

                except sr.UnknownValueError:
                    print("Could not understand the audio, please try again.")
                    logging.warning("Could not understand the audio")
                except sr.RequestError as e:
                    print(f"Speech recognition service error: {e}")
                    logging.error(f"Speech recognition service error: {e}")
                    time.sleep(2)

            except sr.WaitTimeoutError:
                print("No voice detected within timeout, listening again...")
                logging.info("No voice detected within timeout")
                recognizer.adjust_for_ambient_noise(source, duration=2)
            except Exception as e:
                print(f"Unexpected error in voice recognition: {e}")
                logging.error(f"Unexpected error in voice recognition: {e}")
                time.sleep(2)

def start_voice_thread():
    voice_thread = threading.Thread(target=listen_for_voice_commands, daemon=True)
    voice_thread.start()
    return voice_thread

# Modes
mode = "default"
mode_set_time = None
cooldown_seconds = 10

gesture_detected_time = {}
gesture_labels = {0: "Call Nurse", 2: "Need Water", 3: "Call Family"}
gesture_sounds = {0: "call_nurse", 2: "need_water", 3: "call_family"}

cap = cv2.VideoCapture(0)


while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 480))
    if not ret:
        print("Failed to capture frame from camera")
        logging.error("Failed to capture frame from camera")
        break

    # --- Night Mode Auto Detection ---
    if detect_low_light(frame):
        frame = apply_night_mode(frame)
        cv2.putText(frame, "Night Mode", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    else:
        frame = adjust_gamma(frame, gamma=0.5)

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(rgb_frame)
    person_count = 0
    unknown_face_detected = False

    for result in results:
        for box in result.boxes:
            if int(box.cls) == 0:
                person_count += 1
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                person_roi = rgb_frame[y1:y2, x1:x2]
                face_locations = face_recognition.face_locations(person_roi)

                if face_locations:
                    top, right, bottom, left = face_locations[0]
                    top += y1
                    bottom += y1
                    left += x1
                    right += x1
                    face_encoding = face_recognition.face_encodings(rgb_frame, [(top, right, bottom, left)])
                    if face_encoding:
                        matches = face_recognition.compare_faces(known_face_encodings, face_encoding[0])
                        name = "Unknown"
                        if True in matches:
                            name = known_face_names[matches.index(True)]
                        else:
                            unknown_face_detected = True

                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    results_hands = hands.process(rgb_frame)
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            fingers_up = count_fingers(hand_landmarks)
            cv2.putText(frame, f"Fingers: {fingers_up}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if mode_set_time is None or (time.time() - mode_set_time > cooldown_seconds):
                if fingers_up == 5 and mode != "default":
                    mode = "default"
                    mode_set_time = time.time()
                    print("Switched to Default Monitoring Mode")
                    logging.info("Switched to Default Monitoring Mode")

                elif fingers_up == 4 and mode != "gesture":
                    mode = "gesture"
                    mode_set_time = time.time()
                    print("Gesture Mode Activated")
                    logging.info("Gesture Mode Activated")

                elif fingers_up == 1 and mode != "voice":
                    mode = "voice"
                    mode_set_time = time.time()
                    start_voice_thread()
                    print("Voice Recognition Mode Activated")
                    logging.info("Voice Recognition Mode Activated")

            if mode == "gesture" and fingers_up in gesture_labels:
                gesture = gesture_labels[fingers_up]
                cv2.putText(frame, f"Gesture: {gesture}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)

                if fingers_up not in gesture_detected_time:
                    gesture_detected_time[fingers_up] = time.time()
                elif time.time() - gesture_detected_time[fingers_up] > 2:
                    trigger_alarm(f"Gesture: {gesture}", ALARM_SOUNDS[gesture_sounds[fingers_up]])
                    gesture_detected_time.pop(fingers_up)
            else:
                gesture_detected_time.clear()
    else:
        gesture_detected_time.clear()

    cv2.putText(frame, f"People: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Mode: {mode.upper()}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 100), 2)

    if unknown_face_detected:
        print("Unknown face detected!")
        logging.warning("Unknown face detected")
        threading.Thread(target=playsound, args=(ALARM_SOUNDS["unknown_alert"],), daemon=True).start()

    if person_count > 50:
        trigger_alarm(f"Too many people: {person_count}", ALARM_SOUNDS["crowd_alert"])

    cv2.imshow("Patient Room Monitoring", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\sadik\AppData\Local\Programs\Python\Python39\lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename



0: 480x640 1 person, 1 tie, 62.4ms
Speed: 2.4ms preprocess, 62.4ms inference, 85.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.4ms
Speed: 2.3ms preprocess, 13.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.1ms
Speed: 1.1ms preprocess, 9.1ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.8ms
Speed: 1.3ms preprocess, 7.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.9ms
Speed: 1.1ms preprocess, 7.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.2ms
Speed: 2.9ms preprocess, 8.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 14.4ms
Speed: 1.2ms preprocess, 14.4ms inference, 3.6ms postprocess per image at shape (1, 3, 480,

KeyboardInterrupt: 

PermissionError: [Errno 13] Permission denied: 'C:\\windows\\system32\\patient_room_monitoring.log'

In [4]:
import os
from pymongo import MongoClient
import bson  # For handling binary data like images
import datetime

# --- Configuration ---
# Make sure this matches the settings in your main application
MONGO_URI = "mongodb://localhost:27017/"
MONGO_DB_NAME = "patient_monitoring"
MONGO_COLLECTION_NAME = "patients"

# --- Patient Information ---
# This is the data for the new patient you want to add.
# MongoDB uses '_id' as the unique identifier for a document.
patient_id = "PAT001"
patient_name = "John Doe"
patient_age = 76
patient_gender = "Male"
# IMPORTANT: Make sure you have an image file with this name in the same folder as the script.
patient_photo_path =r"C:\Users\sadik\PyCharmMiscProject\tamizh.jpg"

def add_patient_to_db():
    """Connects to MongoDB and inserts a new patient document."""

    # 1. Connect to the database
    try:
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ismaster') # Check if the connection is successful
        db = client[MONGO_DB_NAME]
        collection = db[MONGO_COLLECTION_NAME]
        print(f"Successfully connected to MongoDB database: '{MONGO_DB_NAME}'")
    except Exception as e:
        print(f"Error: Could not connect to MongoDB. {e}")
        return

    # 2. Check if the patient photo exists
    if not os.path.exists(patient_photo_path):
        print(f"Error: Photo file not found at '{patient_photo_path}'. Please provide a valid image file.")
        # Create a placeholder file if it doesn't exist, so the user can replace it.
        with open(patient_photo_path, 'wb') as f:
            f.write(b'') # Write empty bytes
        print(f"A placeholder file has been created. Please replace '{patient_photo_path}' with the patient's actual photo.")
        return

    # 3. Read the image file as binary data
    try:
        with open(patient_photo_path, 'rb') as image_file:
            photo_data = image_file.read()
    except Exception as e:
        print(f"Error reading image file: {e}")
        return

    # 4. Create the patient document
    patient_document = {
        "_id": patient_id,  # Setting a custom ID
        "name": patient_name,
        "age": patient_age,
        "gender": patient_gender,
        "photo": bson.binary.Binary(photo_data), # Store the image data in binary format
        "registration_date": datetime.datetime.utcnow()
    }

    # 5. Insert the document into the collection
    try:
        # Check if a patient with this ID already exists to avoid duplicates
        if collection.find_one({"_id": patient_id}):
            print(f"Patient with ID '{patient_id}' already exists. No new record was added.")
        else:
            collection.insert_one(patient_document)
            print(f"Successfully added patient '{patient_name}' with ID '{patient_id}' to the '{MONGO_COLLECTION_NAME}' collection.")

    except Exception as e:
        print(f"Error inserting document into MongoDB: {e}")
    finally:
        # Close the connection
        client.close()

if __name__ == "__main__":
    add_patient_to_db()


Successfully connected to MongoDB database: 'patient_monitoring'
Successfully added patient 'John Doe' with ID 'PAT001' to the 'patients' collection.


In [5]:
import os
from pymongo import MongoClient
from PIL import Image # Pillow library is great for handling images
import io

# --- Configuration ---
# Make sure this matches the settings in your main application
MONGO_URI = "mongodb://localhost:27017/"
MONGO_DB_NAME = "patient_monitoring"
MONGO_COLLECTION_NAME = "patients"

def fetch_patient_data(patient_id_to_find):
    """Connects to MongoDB, fetches a patient document by its ID, and saves the photo."""

    # 1. Connect to the database
    try:
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ismaster') # Check if the connection is successful
        db = client[MONGO_DB_NAME]
        collection = db[MONGO_COLLECTION_NAME]
        print(f"Successfully connected to MongoDB database: '{MONGO_DB_NAME}'")
    except Exception as e:
        print(f"Error: Could not connect to MongoDB. {e}")
        return

    # 2. Fetch the document from the collection
    try:
        print(f"\nSearching for patient with ID: {patient_id_to_find}...")
        patient_document = collection.find_one({"_id": patient_id_to_find})

        if patient_document:
            print("--- Patient Found ---")
            print(f"ID: {patient_document.get('_id')}")
            print(f"Name: {patient_document.get('name')}")
            print(f"Age: {patient_document.get('age')}")
            print(f"Gender: {patient_document.get('gender')}")
            print(f"Registration Date: {patient_document.get('registration_date')}")

            # 3. Handle and save the photo
            photo_data = patient_document.get('photo')
            if photo_data and isinstance(photo_data, bytes):
                try:
                    # Use Pillow to open the image from the binary data
                    image = Image.open(io.BytesIO(photo_data))

                    # Define the output path for the saved photo
                    output_filename = f"retrieved_{patient_id_to_find}.jpg"

                    # Save the image to a file
                    image.save(output_filename)
                    print(f"\nSuccessfully saved patient photo as '{output_filename}'")

                    # Optional: Display the image
                    # image.show()

                except Exception as img_e:
                    print(f"Error processing image data: {img_e}")
            else:
                print("No photo data found for this patient.")

        else:
            print("-----------------------")
            print("Patient not found.")
            print("-----------------------")

    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
    finally:
        # 4. Close the connection
        client.close()
        print("\nDatabase connection closed.")


if __name__ == "__main__":
    # The ID of the patient you want to retrieve
    patient_id_to_fetch = "PAT001"
    fetch_patient_data(patient_id_to_fetch)


Successfully connected to MongoDB database: 'patient_monitoring'

Searching for patient with ID: PAT001...
--- Patient Found ---
ID: PAT001
Name: John Doe
Age: 76
Gender: Male
Registration Date: 2025-08-15 20:24:21.619000
Error processing image data: [Errno 13] Permission denied: 'retrieved_PAT001.jpg'

Database connection closed.
